In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd

In [2]:
#gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?title=Ukraine Russia&begin_date=2022-02-24&end_date=2022-06-23&orderByTime=yes&page_no={}'
#gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?textPage=Ukraine&begin_date=2022-02-24&end_date=2022-06-23&orderByTime=yes&page_no={}'
gt_base_url = 'https://search.globaltimes.cn/SearchCtrl?textPage=Ukraine&begin_date=2021-08-24&end_date=2022-02-23&orderByTime=yes&page_no={}'

In [3]:
links = []
for page_number in range(1, 22):  #loop from page 1 to 165
    gt_url = gt_base_url.format(page_number)
    #print(gt_url)
    print("Processing page:", page_number)
    
    r = requests.get(gt_url)
    b = BeautifulSoup(r.content, 'html.parser')

    for news in b.findAll('div', {'class': 'span9'}):
        link = news.find('a')['href']
        links.append(link)

print("Total links collected:", len(links))
print("Sample links:", links[:5]) 

Processing page: 1
Processing page: 2
Processing page: 3
Processing page: 4
Processing page: 5
Processing page: 6
Processing page: 7
Processing page: 8
Processing page: 9
Processing page: 10
Processing page: 11
Processing page: 12
Processing page: 13
Processing page: 14
Processing page: 15
Processing page: 16
Processing page: 17
Processing page: 18
Processing page: 19
Processing page: 20
Processing page: 21
Total links collected: 203
Sample links: ['https://www.globaltimes.cn/page/202202/1252930.shtml', 'https://www.globaltimes.cn/page/202202/1252936.shtml', 'https://www.globaltimes.cn/page/202202/1252938.shtml', 'https://www.globaltimes.cn/page/202202/1252939.shtml', 'https://www.globaltimes.cn/page/202202/1252940.shtml']


In [4]:
#initialize webdriver
s = Service(r"C:\Program Files (x86)\chromedriver.exe")
driver = webdriver.Chrome(service=s)

article_titles = []
authors = []
published_times = []
article_texts = []

#loop through each link
for link in links:
    #navigate to the page containing the article
    driver.get(link)

    #extract the HTML content of the page
    html_content = driver.page_source

    #parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    #find article title
    article_title_elem = soup.find("div", class_="article_title")
    article_title = article_title_elem.text.strip() if article_title_elem else "Title not found"
    article_titles.append(article_title)

    #find author and published time
    author_published_time = soup.find("div", class_="author_share_left")
    author = author_published_time.find("span", class_="byline").text.strip() if author_published_time else "Author not found"
    published_time = author_published_time.find("span", class_="pub_time").text.strip() if author_published_time else "Published time not found"
    authors.append(author)
    published_times.append(published_time)

    #find article text
    article_content_elem = soup.find("div", class_="article_content")
    article_text = article_content_elem.text.strip() if article_content_elem else "Article text not found"
    article_texts.append(article_text)

#close webdriver
driver.quit()

In [5]:
#save to csv file
data = {
    "article_title": article_titles,
    "author": authors,
    "published_time": published_times,
    "article_text": article_texts
}

df = pd.DataFrame(data)
df.to_csv("ukraine_textcontain_before.csv", index=False)